In [1]:
import numpy as np
import pandas as pd
import torch
import pickle
import re
from itertools import chain
from collections import Counter
import torch.nn as nn
import glob
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
import transformers
from transformers import AdamW
from transformers import DistilBertTokenizer, DistilBertModel
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, Dataset, random_split
from tqdm import tqdm
import time

In [2]:
def random_text_splitter(text):

    import random

    """
    Random breaks up a text into an X amount of sentences. 
    The output sentences consist of a minimum of 10 sentences.
    """

    # Split text
    words = text.split()
    # Get the amount of words
    word_amount = len(words)
    if word_amount <= 10:
        return [text]

    # Create counter
    remaining_word_amount = word_amount
    # Init list
    parts = []
    # While words remaining
    while remaining_word_amount > 0:
        if len(words) < 10:
            # Add last part if less then 10
            parts[-1] = parts[-1] + ' '.join(words)
            # exit
            remaining_word_amount = 0
        # Generate random int
        randint = random.randint(10, word_amount)
        # Append to list 
        parts.append(' '.join(words[:randint]))
        # Delete previous selection
        words = words[randint:]
        # Update counter
        remaining_word_amount -= randint

    return parts

In [3]:
samples = []

# Load the pickle list
datalist = glob.glob('../data/processed/train*.pkl')
# Init list
data_values = []
# Loop over the pickles
for data in datalist:
    # Open the pickles
    datadict = pickle.load(open(data, 'rb'))
    # Undict and append
    data_values += (list(chain.from_iterable(datadict.values())))

# Drop double values 
data_values = list(set(data_values))

nested_values = [[tuple([1, span]) if text[0] == 1 else tuple([0, span]) 
                  for span in random_text_splitter(text[1])] 
                 for text in data_values]

samples += list(chain.from_iterable(nested_values))

In [6]:
with open('../data/processed/COLAB.pkl', 'wb') as f:
    pickle.dump(samples, f)    

In [4]:
len(samples)

2258184

In [ ]:
nested_values[0:10]